In [1]:
# RNN(Recurrent Neural Network)

# torch.nn.RNN(*args, **kwargs) # *args는 튜플, **kargs는 딕셔너리로 처리

# Parameters

# input_size : 입력 사이즈
# hidden_size : 은닉층 사이즈
# num_layers : RNN의 은닉층 레이어 갯수
# nonlinearity : 비선형 활성화 함수 (tanh, relu), default : tanh
# bias : 편향 활성화 여부, default : True
# batch_first : True일시 Output_size : (batch,seq,feature), default : False
# dropout : 드롭아웃 비율 설정, default : 0
# bidirectional : True=> 양방향 RNN, default : False

In [84]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

sentences = ['i like dog', 'i love coffee','i hate milk', "you like cat", "you love milk", "you hate coffee"]

# 전처리

a = " ".join(sentences)
word_list  = a.split()  # 단어별 토크나이즈 
word_set = set(word_list)       # set로 바꿔서 중복 없애기
word2id = {w:i for i,w in enumerate(word_set)}  # 딕셔너리 => 워드 : 워드ID
id2word = {i:w for i,w in enumerate(word_set)}  # 딕서너리 => 워드ID : 워드
n_class = len(word2id)    # 총 단어수 


# RNN 파라미터

batch_size = len(sentences)
# n_step = 2 # 학습하려고 하는 문장 길이 -1   # 지금 문장길이 : 3
n_hidden = 5    # 은닉층 레이어 개수

def make_batch(sentences):
    input_batch = []
    target_batch = []
    
    for sentence in sentences:
        word = sentence.split()
        input_id = [word2id[n] for n in word[:-1]]   # 마지막 글자는 뺀다.
        target_id = word2id[word[-1]]    # target은 마지막 단어
        print(input_id)
        input_batch.append(np.eye(n_class)[input_id])
        target_batch.append(target_id) # 문장별 마지막 단어만 넣는다.
        
    return input_batch, target_batch

input_batch, target_batch = make_batch(sentences)

# torch.tensor는 되지만, torch.Tensor는 안 된다.
input_batch = torch.tensor(input_batch, dtype=torch.float, requires_grad=True)  # float형만 requires_grad가 가능하다.
target_batch = torch.tensor(target_batch, dtype=torch.int64)    # list -> tensor 형 변환, long type 이여야 하기때문에 int64
# print(input_batch)

# RNN
def init_weights(m):
    if isinstance(m, nn.Linear):    # Linear라면 사비에르 초기화, 편향 :0
        nn.init.xavier_normal_(m.weight)    # m.weight.data
        m.bias.data.fill_(0)

nonlinearity = torch.nn.Tanh()
class TextRNN(nn.Module):
    def __init__(self):
        super(TextRNN,self).__init__()
        # RNN input은 원핫 벡터로 어떠한 단어가 들어오는지 알려준다.
        self.rnn = nn.RNN(input_size=n_class, hidden_size=n_hidden,dropout = 0)   # dropout >0 : num_layers는 1보다 큰걸 권장한다.
        self.layer = nn.Linear(n_hidden,n_class)
        # self.softmax = nn.Softmax(dim=1)    # 분류이므로 softmax 계층이 필요하다.
        self.layer.apply(init_weights)  # layer계층에 적용시킨다.

                
    def forward(self,hidden,x):
        x = x.transpose(0,1)       # x: N,L,H -> L,N,H => batch_first=False일 때 쓰는 꼴 // True일 때는 N,L,H
        outputs, hidden = self.rnn(x,hidden)
        output = outputs[-1]    # output의 hidden layer만 필요하다.
        y = self.layer(output)  # 클래스 분류
        return y
    
    
# Trainging

model = TextRNN()
criterion = nn.CrossEntropyLoss()   # 분류이므로 (소프트맥스까지 취한다.)
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(500):
    hidden = torch.zeros(1,batch_size,n_hidden,requires_grad=True)  # hidden 초기화
    pred = model(hidden,input_batch)
    loss = criterion(pred,target_batch) 

    if (epoch + 1) % 100 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# test
input = [sentence.split()[:2] for sentence in sentences]    # 2단어까지만 넣기
test_data = [[word2id[i], word2id[j]] for i,j in input] 
test_list = []
for data in test_data:
    test_list.append(np.eye(n_class)[list(data)])   # data를 원핫인코딩

test_list = torch.tensor(test_list, dtype=torch.float)  # list -> tensor
print(test_list)
# test이므로 requires_grad=False로 둔다.
model.eval()
hidden = torch.zeros(1,batch_size,n_hidden,dtype=torch.float,requires_grad=False)
pred = model(hidden,test_list) # model의 인풋은 원핫인코딩과 초기화된 hidden이다.
_, pred_id = torch.max(pred,1)
pred_id = pred_id.tolist()
pred_words = [id2word[id] for id in pred_id]
for sentence, pred_word in zip(input, pred_words):
    print(sentence[0],sentence[1],pred_word)
# why? x.transpos(0,1) 
# outputs.shape가 2,6,2 일까?

[6, 7]
[6, 4]
[6, 1]
[3, 7]
[3, 4]
[3, 1]
Epoch: 0100 cost = 0.387569
Epoch: 0200 cost = 0.075454
Epoch: 0300 cost = 0.030855
Epoch: 0400 cost = 0.017526
Epoch: 0500 cost = 0.011607
tensor([[[0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0.]],

        [[0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0.]],

        [[0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0.]],

        [[0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0.]]])
i like dog
i love coffee
i hate milk
you like cat
you love milk
you hate coffee


In [81]:
# 딕셔너리에 tensor는 들어가지 않는다.
dic = {0:1, 1:2}
a = torch.tensor([0,1])
for aa in a:
    print(dic[aa])

KeyError: tensor(0)

In [70]:
a = torch.tensor([[0.3,0.1],[0.1,0.3]])
torch.max(a,1)

torch.return_types.max(
values=tensor([0.3000, 0.3000]),
indices=tensor([0, 1]))

In [6]:
sentences = ['i like a dog', 'i love coffee','i hate milk', "you like cat", "you love milk", "you hate coffee"]
print(" ".join(sentences)) #리스트안의 문자열 통합
a = " ".join(sentences)
print(a.split())    # 띄어쓰기 기준 리스트화 // 단어별토크나이즈
word_list  = a.split()
word_dict = {w: i for i,w in enumerate(word_list)}  # 딕셔너리 => 워드 : 워드ID
number_dict = {i:w for i,w in enumerate(word_list)}  # 딕서너리 => 워드ID : 워드
print(word_dict,number_dict)

i like a dog i love coffee i hate milk you like cat you love milk you hate coffee
['i', 'like', 'a', 'dog', 'i', 'love', 'coffee', 'i', 'hate', 'milk', 'you', 'like', 'cat', 'you', 'love', 'milk', 'you', 'hate', 'coffee']
{'i': 7, 'like': 11, 'a': 2, 'dog': 3, 'love': 14, 'coffee': 18, 'hate': 17, 'milk': 15, 'you': 16, 'cat': 12} {0: 'i', 1: 'like', 2: 'a', 3: 'dog', 4: 'i', 5: 'love', 6: 'coffee', 7: 'i', 8: 'hate', 9: 'milk', 10: 'you', 11: 'like', 12: 'cat', 13: 'you', 14: 'love', 15: 'milk', 16: 'you', 17: 'hate', 18: 'coffee'}


In [8]:
import torch
torch.randn(3,5)

c:\Users\DILAB\anaconda3\envs\DL\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([[ 0.3501, -1.0705,  0.3398, -0.5640, -0.1408],
        [-1.4608, -0.0841,  0.2765, -0.4620, -0.0844],
        [ 0.2074, -0.1847,  0.4997, -0.9865, -0.2218]])

In [17]:
import numpy as np
a = torch.randint(5,(5,))
print(a)
np.eye(5)[a]    # 원핫벡터

tensor([2, 0, 3, 3, 2])


array([[0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])

In [27]:
sentences = ['i like a dog', 'i love coffee','i hate milk', "you like cat", "you love milk", "you hate coffee"]
word2id = {w: i for i,w in enumerate(word_list)}  # 딕셔너리 => 워드 : 워드ID
id2word = {i:w for i,w in enumerate(word_list)}  # 딕서너리 => 워드ID : 워드
input = [sentence.split()[:2] for sentence in sentences]    # 2단어까지만 넣기

print(input)
test_data = [[word2id[i], word2id[j]] for i,j in input] 
test_data

[['i', 'like'], ['i', 'love'], ['i', 'hate'], ['you', 'like'], ['you', 'love'], ['you', 'hate']]


[[7, 11], [7, 14], [7, 17], [16, 11], [16, 14], [16, 17]]

In [57]:
np.eye(5)[torch.tensor([1,3])]

array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.]])

In [58]:
np.eye(5)[[1,3]]

array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.]])